# Calcul et Visualisation de la HGL (version corrigée)
Ce notebook reprend le calcul et la visualisation de la HGL avec gestion robuste des valeurs manquantes.

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
g = 9.81  # gravité
stations = {
    'Head': {'elevation_m': 650, 'distance_km': 0},
    'PMS1': {'elevation_m': 680, 'distance_km': 45},
    'VANNE': {'elevation_m': 515, 'distance_km': 68},
    'PMS2': {'elevation_m': 360, 'distance_km': 101},
    'PMS3': {'elevation_m': 110, 'distance_km': 130},
    'PMS4': {'elevation_m': 143, 'distance_km': 162},
    'Terminal': {'elevation_m': 60, 'distance_km': 187}
}

In [25]:
# Charger le dataset fusionné
merged_df = pd.read_csv('./new_dataset/Merged_All_Stations_updated.csv', sep=',', decimal='.', engine='python')
for col in ['Density_HS_Average', 'Density_PMS1', 'Density_VANNE', 'Density_PMS2', 'Density_PMS3', 'Density_PMS4', 'Density_T',
            'Pressure_HS_Average', 'Pressure_PMS1', 'Pressure_VANNE', 'Pressure_PMS2', 'Pressure_PMS3', 'Pressure_PMS4', 'Pressure_T']:
    if col in merged_df.columns:
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

In [26]:
river_crossings = [
    {'start_km': 60, 'end_km': 65, 'water_elev_m': 1150},
    {'start_km': 150, 'end_km': 152, 'water_elev_m': 1080},
]
def adjust_elevation(distance, base_elevation, river_crossings):
    for river in river_crossings:
        if river['start_km'] <= distance <= river['end_km']:
            return river['water_elev_m']
    return base_elevation

In [27]:
def calculate_HGL(row, station):
    stations_columns = {
        'Head': 'HS',
        'PMS1': 'PMS1',
        'VANNE': 'VANNE',
        'PMS2': 'PMS2',
        'PMS3': 'PMS3',
        'PMS4': 'PMS4',
        'Terminal': 'T'
    }
    prefix = stations_columns[station]
    P_col = f'Pressure_{prefix}' if prefix != 'HS' else 'Pressure_HS_Average'
    rho_col = f'Density_{prefix}' if prefix != 'HS' else 'Density_HS_Average'
    P = row.get(P_col, None)
    rho = row.get(rho_col, None)
    if pd.isna(P) or pd.isna(rho):
        return None
    P = P * 1e5
    dist = stations[station]['distance_km']
    z = stations[station]['elevation_m']
    z_adjusted = adjust_elevation(dist, z, river_crossings)
    return ((P * 0.10197) / (rho * g)) + z_adjusted

In [28]:
for station in stations:
    merged_df[f'HGL_{station}'] = merged_df.apply(lambda row: calculate_HGL(row, station), axis=1)

In [29]:
merged_df.to_csv("./new_dataset/HGL_dataset.csv", index=False)

In [31]:
import tensorflow as tf
print(f"model_utils.py - TensorFlow Version: {tf.__version__}")
print(f"model_utils.py - Keras Version: {tf.keras.__version__}")

model_utils.py - TensorFlow Version: 2.16.2
model_utils.py - Keras Version: 3.10.0
